In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from preprocessor import Preprocessor
import random
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import combinations

In [2]:
train_preprocessor = Preprocessor()

raw_data = pd.read_csv("../../data/wikipedia.csv")
# clean out where text is NaN
raw_data = raw_data[raw_data.text.notna()]

In [3]:
x = []
y = []
for row in raw_data.text:
    # get words TODO only relevant words
    words = train_preprocessor.normalize_words(row)

    for (a, b) in combinations(words,2):
        x.append(a)
        y.append(b)

 Wesen mental Zustand Emotion Qualia untersuchen Weiterhin befasst Gebiet Beurteilung verschieden Bewusstseinszustände Überlegung künstlich Intelligenz Identität Problem möglich physisch Tod ontologisch epistemologische semantisch
bekennen Vertreter Problemfelder Gottfried Wilhelm Leibniz Baruch de Spinoza Alan Turing Hilary Putnam John Searle Kim Donald Davidson philosophisch Bedeutung Kontext Buddhismus ausgearbeitet Theorie
modern philosophisch Anthropologie befasst Wesen Mensch vornehmlich Individuum Mensch betrieben dialektische Selbstreflexion gleichzeitig Innen- Außenperspektive aufweisen Mensch Einbeziehung wichtig Erkenntnis untersuchen
Wesen Mensch Rätsel Stellung Kosmos Verhältnis Kultur Natur Vereinzelung Vergemeinschaftung Problem Geschlechtlichkeit Rolle lieben Tod Grundfragen philosophisch Anthropologie Mensch Natur böse Gewalt Leid zwingen menschlich Existenz gehören Leben Sinn all Problem Disziplin untersuchen grundsätzliche menschliche Bedürfnis Fähigkeit Selbstverwir

In [4]:
len(x)

173887

In [5]:
features = tf.constant(x, shape=(1,len(x),300))

labels = tf.constant(y, shape=(1,len(y),300))

#train_dataset = tf.data.Dataset.from_tensor_slices((features,labels))
features_dataset = tf.data.Dataset.from_tensor_slices(features)
labels_dataset = tf.data.Dataset.from_tensor_slices(labels)

train_dataset = tf.data.Dataset.zip((features_dataset, labels_dataset))

In [6]:
train_dataset

<ZipDataset shapes: ((173887, 300), (173887, 300)), types: (tf.float32, tf.float32)>

In [7]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(100, activation=tf.nn.relu, input_shape=(300,)),
  tf.keras.layers.Dense(100, activation=tf.nn.relu),
  tf.keras.layers.Dense(300)
])

In [8]:
loss_object = tf.keras.losses.MeanSquaredError()

# def get_random_result(label, labels, predictions, predictions_cache):
#   label = label[0]
#   # TODO: Check if the index of the x value we are testing is not exactly the same as the one we are returning as y
#   if(label not in predictions_cache):
#     predictions_cache[label] = []
#     for idx2, value in enumerate(labels.numpy()):
#         if(value == label):
#             predictions_cache[label].append(predictions[idx2])
#   return predictions_cache[label][random.randint(0,len(predictions_cache[label])-1)]

# def get_new_y(predictions, labels):
#   predictions_cache = {}
#   new_y = []
#   for idx, val in enumerate(predictions.numpy()):
#       label = labels[idx].numpy()
#       new_y.append(get_random_result(label, labels, predictions, predictions_cache))
#   return new_y

def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)
  #new_y = get_new_y(y_, y)

  return loss_object(y_true=y, y_pred=y_)

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [9]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [25]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []
num_epochs = 2000
import time
for epoch in range(num_epochs):
  start = time.time()
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.MeanSquaredError()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(x, training=True)
    #new_y = get_new_y(predictions, y)
    epoch_accuracy.update_state(y, predictions)

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())
  end = time.time()
  print("Epoch {:03d}: Loss: {:.3f}, MAE: {:.3}, time: {:.8f}".format(epoch,epoch_loss_avg.result(),epoch_accuracy.result(), end - start))

, time: 1.02718663
Epoch 1623: Loss: 4.631, MAE: 4.63, time: 1.06734300
Epoch 1624: Loss: 4.631, MAE: 4.63, time: 1.08012938
Epoch 1625: Loss: 4.631, MAE: 4.63, time: 1.03348231
Epoch 1626: Loss: 4.631, MAE: 4.63, time: 1.03446698
Epoch 1627: Loss: 4.631, MAE: 4.63, time: 1.02221584
Epoch 1628: Loss: 4.631, MAE: 4.63, time: 1.07991672
Epoch 1629: Loss: 4.631, MAE: 4.63, time: 1.05967784
Epoch 1630: Loss: 4.631, MAE: 4.63, time: 1.06472111
Epoch 1631: Loss: 4.631, MAE: 4.63, time: 1.01358771
Epoch 1632: Loss: 4.631, MAE: 4.63, time: 1.02380204
Epoch 1633: Loss: 4.630, MAE: 4.63, time: 1.01523590
Epoch 1634: Loss: 4.630, MAE: 4.63, time: 1.02855515
Epoch 1635: Loss: 4.630, MAE: 4.63, time: 1.03223062
Epoch 1636: Loss: 4.630, MAE: 4.63, time: 1.06196833
Epoch 1637: Loss: 4.630, MAE: 4.63, time: 1.03059793
Epoch 1638: Loss: 4.630, MAE: 4.63, time: 1.06705832
Epoch 1639: Loss: 4.630, MAE: 4.63, time: 1.07366085
Epoch 1640: Loss: 4.630, MAE: 4.63, time: 1.06841135
Epoch 1641: Loss: 4.630, MA

In [11]:
a = train_preprocessor.vectorize("Brot")
b = train_preprocessor.vectorize("häkeln")
c = train_preprocessor.vectorize("Mehl")
d = train_preprocessor.vectorize("Luftmasche")

trained = model(tf.constant([a,b,c,d]), training=False)
diff1 = np.linalg.norm(trained[0]-trained[2])
diff2 = np.linalg.norm(trained[1]-trained[3])
diff3 = np.linalg.norm(trained[0]-trained[3])
diff4 = np.linalg.norm(trained[1]-trained[2])
print("Should be low")
print(diff1)
print(diff2)
print("Should be higher")
print(diff3)
print(diff4)

diff1 = np.linalg.norm(a-c)
diff2 = np.linalg.norm(b-d)
diff3 = np.linalg.norm(a-d)
diff4 = np.linalg.norm(b-c)
print("Should be low")
print(diff1)
print(diff2)
print("Should be higher")
print(diff3)
print(diff4)

Should be low
7.693727
8.411027
Should be higher
10.316896
7.7869754
Should be low
31.130438
32.286346
Should be higher
43.48296
46.15848


In [28]:
import importlib
import preprocessor
importlib.reload(preprocessor)
train_preprocessor = preprocessor.Preprocessor()
def test(text):
    word_vectors = train_preprocessor.normalize_words(text)
    predicted_vectors = model(tf.constant(word_vectors), training=False)
    return np.average(predicted_vectors, axis=0)

# a = "Brot (ahd. prôt, von urgerm. *brauda-) ist ein traditionelles Nahrungsmittel, das aus einem Teig aus gemahlenem Getreide (Mehl), Wasser, einem Triebmittel und meist weiteren Zutaten gebacken wird. Brot zählt zu den Grundnahrungsmitteln."
# b = "Eine enge Gangabstufung ist auch für Transportarbeiten günstig, da das Verhältnis von Leistung und Gesamtgewicht des Zuges bei Traktoren häufig geringer ist als bei Lastkraftwagen."
a = "Getreide"
b = "Luftmasche"
query = "Brot"
a = test(a)
b = test(b)
query = test(query)


diff1 = np.linalg.norm(query - a)
diff2 = np.linalg.norm(query - b)
diff3 = np.linalg.norm(a - b)

print(diff1)
print(diff2)
print(diff3)

4.796025
4.8885045
3.2877345


In [23]:
query = np.array([1,1,1])
a = np.array([-1,-1,-1])
print(query - a)
print(np.linalg.norm(query - a))

tmp = query - a
sum_squared = np.dot(tmp.T, tmp)
result = np.sqrt(sum_squared)
print(result)

[2 2 2]
3.4641016151377544
3.4641016151377544
